In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neptune
from models import create_compound_model_from_info, create_pytorch_model_from_info, MultiHead
import json
import torch
import os
from utils_neptune import check_if_path_in_struc, get_sub_struc_from_path

NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='


In [2]:
run_id = 'RCC-2925'

task_id = 'Optimized_NIVO-OS ADV EVER-OS__133_finetune'
local_dir = os.path.expanduser('~/saved_models')
model_dir = f'{local_dir}/{run_id}/{task_id}'
components_dir = f'{local_dir}/{run_id}/{task_id}/components'
os.makedirs(components_dir, exist_ok=True)

In [3]:

run = neptune.init_run(project='revivemed/RCC',
    api_token= NEPTUNE_API_TOKEN,
    with_id=run_id,
    mode="read-only")   
run_struc= run.get_structure()

substruc = get_sub_struc_from_path(run_struc,f'{task_id}/models')
for key in substruc.keys():
    if 'info' in key:
        run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.json')
    elif 'state' in key:
        run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.pt')

run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925/metadata
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925/metadata


In [3]:
# load the relavent encoder info and head info
head_key = 'Cox_NIVO'
model_files = os.listdir(components_dir)

encoder_info = None
head_info = None
encoder_state = None
head_state = None

for f in model_files:
    if ('encoder' in f):
        if 'info' in f:
            encoder_info = json.load(open(f'{components_dir}/{f}'))
        elif 'state' in f:
            encoder_state = torch.load(f'{components_dir}/{f}')
    if (head_key in f):
        if 'info' in f:
            head_info = json.load(open(f'{components_dir}/{f}'))
            # head_name = f.replace('_info.json', '')
        elif 'state' in f:
            head_state = torch.load(f'{components_dir}/{f}')
            
            
if encoder_info is not None and head_info is not None:            
    model = create_compound_model_from_info(encoder_info=encoder_info, 
                                            head_info= head_info,
                                            encoder_state_dict=encoder_state,
                                            head_state_dict=head_state)
    
    model.save_info(model_dir, f'VAE {head_key} info.json')
    model.save_state_to_path(model_dir, f'VAE {head_key} state.pt')

    skmodel = create_pytorch_model_from_info(full_model=model)

In [4]:
data_dir = '/app/finetune_data'
X_data = pd.read_csv(f'{data_dir}/X_finetune_test.csv',index_col=0)
y_data = pd.read_csv(f'{data_dir}/y_finetune_test.csv',index_col=0)

In [7]:
y_preds = skmodel.predict(X_data.to_numpy())

In [6]:
skmodel.score(X_data.to_numpy(),y_data[['OS','OS_Event']].to_numpy())

{'Concordance Index': 0.6098095902701625}

In [9]:
skmodel.score(X_data.to_numpy(),y_data[['NIVO OS','OS_Event']].to_numpy())

{'Concordance Index': 0.6371973587674248}

In [10]:
skmodel.score(X_data.to_numpy(),y_data[['EVER OS','OS_Event']].to_numpy())

{'Concordance Index': 0.5876379690949227}

In [ ]:
def generate_survival_report(task_id,pretrained=True):
    if ('finetune' in task_id) or ('randinit' in task_id):
        finetune_id = task_id
    else:
        if pretrained:
            finetune_id = task_id + '_finetune'
        else:
            finetune_id = task_id + '_randinit'

            

In [ ]:
def generate_survival_report(desc_str,head_key,pretrained=True,data_dir=None):
    if data_dir is None:
        data_dir = '/app/finetune_data'
    
    if ('finetune' in desc_str) or ('randinit' in desc_str):
        task_id = desc_str
    else:
        if pretrained:
            task_id = desc_str + '_finetune'
        else:
            task_id = desc_str + '_randinit'

    local_dir = os.path.expanduser('~/saved_models')
    model_dir = f'{local_dir}/{run_id}/{task_id}'
    components_dir = f'{local_dir}/{run_id}/{task_id}/components'
    os.makedirs(components_dir, exist_ok=True)
    model_files = os.listdir(components_dir)
    
    if (len(model_files) < 4):

        run = neptune.init_run(project='revivemed/RCC',
            api_token= NEPTUNE_API_TOKEN,
            with_id=run_id,
            mode="read-only")   
        run_struc= run.get_structure()

        substruc = get_sub_struc_from_path(run_struc,f'{task_id}/models')
        for key in substruc.keys():
            if 'info' in key:
                run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.json')
            elif 'state' in key:
                run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.pt')

        run.stop()
        model_files = os.listdir(components_dir)


    encoder_info = None
    head_info = None
    encoder_state = None
    head_state = None

    for f in model_files:
        if ('encoder' in f):
            if 'info' in f:
                encoder_info = json.load(open(f'{components_dir}/{f}'))
            elif 'state' in f:
                encoder_state = torch.load(f'{components_dir}/{f}')
        if (head_key in f):
            if 'info' in f:
                head_info = json.load(open(f'{components_dir}/{f}'))
                # head_name = f.replace('_info.json', '')
            elif 'state' in f:
                head_state = torch.load(f'{components_dir}/{f}')
                
                
    if encoder_info is not None and head_info is not None:            
        model = create_compound_model_from_info(encoder_info=encoder_info, 
                                                head_info= head_info,
                                                encoder_state_dict=encoder_state,
                                                head_state_dict=head_state)
        
        model.save_info(model_dir, f'VAE {head_key} info.json')
        model.save_state_to_path(model_dir, f'VAE {head_key} state.pt')

        skmodel = create_pytorch_model_from_info(full_model=model)


    X_data = pd.read_csv(f'{data_dir}/X_finetune_test.csv',index_col=0)
    y_data = pd.read_csv(f'{data_dir}/y_finetune_test.csv',index_col=0)


    res_dict = {}

    res_dict['both-OS'] = skmodel.score(X_data.to_numpy(),y_data[['OS','OS_Event']].to_numpy())['Concordance Index']

    res_dict['NIVO-OS'] = skmodel.score(X_data.to_numpy(),y_data[['NIVO OS','OS_Event']].to_numpy())['Concordance Index']
    res_dict['EVER-OS'] = skmodel.score(X_data.to_numpy(),y_data[['EVER OS','OS_Event']].to_numpy())['Concordance Index']

    return res_dict